In [2]:
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument, Doc2VecVocab, LabeledSentence
from gensim.utils import simple_preprocess
import numpy as np
import pandas as pd
import random
import string
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup


In [3]:
data = pd.read_csv('raw/raw_wine_data.csv', sep=',', encoding='utf-8')


In [4]:
varieties = data.variety.unique()

In [5]:
clean_varieties = []
for i in varieties:
    clean_varieties.append(str(i))

In [6]:
sorted(clean_varieties)

['Abouriou',
 'Agiorgitiko',
 'Aglianico',
 'Aidani',
 'Airen',
 'Albana',
 'Albanello',
 'Albariño',
 'Albarossa',
 'Aleatico',
 'Alfrocheiro',
 'Alicante',
 'Alicante Bouschet',
 'Aligoté',
 'Alsace white blend',
 'Altesse',
 'Alvarelhão',
 'Alvarinho',
 'Alvarinho-Chardonnay',
 'Ansonica',
 'Antão Vaz',
 'Apple',
 'Aragonez',
 'Aragonês',
 'Argaman',
 'Arinto',
 'Arneis',
 'Asprinio',
 'Assyrtico',
 'Assyrtiko',
 'Athiri',
 'Austrian Red Blend',
 'Austrian white blend',
 'Auxerrois',
 'Avesso',
 'Azal',
 'Babić',
 'Babosa Negro',
 'Bacchus',
 'Baco Noir',
 'Baga',
 'Baga-Touriga Nacional',
 'Barbera',
 'Barbera-Nebbiolo',
 'Bastardo',
 'Biancale',
 'Biancolella',
 'Biancu Gentile',
 'Bical',
 'Black Monukka',
 'Black Muscat',
 'Blanc du Bois',
 'Blatina',
 'Blauburger',
 'Blauburgunder',
 'Blauer Portugieser',
 'Blaufränkisch',
 'Bobal',
 'Bobal-Cabernet Sauvignon',
 'Bombino Bianco',
 'Bombino Nero',
 'Bonarda',
 'Bordeaux-style Red Blend',
 'Bordeaux-style White Blend',
 'Bovale',

In [7]:
variety_dict = {}
print(random.choice(clean_varieties))

Encruzado


In [8]:
leftovers = []
for i in clean_varieties:
    lower_cased = i.lower()
    if 'red' in lower_cased or 'noir' in lower_cased:
        variety_dict[i] = 'red'
    elif 'white' in lower_cased or 'blanc' in lower_cased or "gewürztraminer" in lower_cased:
        variety_dict[i] = 'white'
    elif 'rosé' in lower_cased:
        variety_dict[i] = 'rosé'
    elif 'sparkling' in lower_cased:
        variety_dict[i] = 'sparkling'
    elif 'cabernet' in lower_cased or 'malbec' in lower_cased:
        variety_dict[i] = 'red'
    else:
        leftovers.append(i)

In [9]:
len(leftovers)

579

In [10]:
variety_dict

{'Alsace white blend': 'white',
 'Austrian Red Blend': 'red',
 'Austrian white blend': 'white',
 'Baco Noir': 'red',
 'Blanc du Bois': 'white',
 'Bobal-Cabernet Sauvignon': 'red',
 'Bordeaux-style Red Blend': 'red',
 'Bordeaux-style White Blend': 'white',
 'Cabernet': 'red',
 'Cabernet Blend': 'red',
 'Cabernet Franc': 'red',
 'Cabernet Franc-Cabernet Sauvignon': 'red',
 'Cabernet Franc-Carmenère': 'red',
 'Cabernet Franc-Lemberger': 'red',
 'Cabernet Franc-Malbec': 'red',
 'Cabernet Franc-Merlot': 'red',
 'Cabernet Merlot': 'red',
 'Cabernet Moravia': 'red',
 'Cabernet Pfeffer': 'red',
 'Cabernet Sauvignon': 'red',
 'Cabernet Sauvignon Grenache': 'red',
 'Cabernet Sauvignon-Barbera': 'red',
 'Cabernet Sauvignon-Cabernet Franc': 'red',
 'Cabernet Sauvignon-Carmenère': 'red',
 'Cabernet Sauvignon-Malbec': 'red',
 'Cabernet Sauvignon-Merlot': 'red',
 'Cabernet Sauvignon-Merlot-Shiraz': 'red',
 'Cabernet Sauvignon-Sangiovese': 'red',
 'Cabernet Sauvignon-Shiraz': 'red',
 'Cabernet Sauvign

In [11]:
sorted(leftovers)

['Abouriou',
 'Agiorgitiko',
 'Aglianico',
 'Aidani',
 'Airen',
 'Albana',
 'Albanello',
 'Albariño',
 'Albarossa',
 'Aleatico',
 'Alfrocheiro',
 'Alicante',
 'Alicante Bouschet',
 'Aligoté',
 'Altesse',
 'Alvarelhão',
 'Alvarinho',
 'Alvarinho-Chardonnay',
 'Ansonica',
 'Antão Vaz',
 'Apple',
 'Aragonez',
 'Aragonês',
 'Argaman',
 'Arinto',
 'Arneis',
 'Asprinio',
 'Assyrtico',
 'Assyrtiko',
 'Athiri',
 'Auxerrois',
 'Avesso',
 'Azal',
 'Babić',
 'Babosa Negro',
 'Bacchus',
 'Baga',
 'Baga-Touriga Nacional',
 'Barbera',
 'Barbera-Nebbiolo',
 'Bastardo',
 'Biancale',
 'Biancolella',
 'Biancu Gentile',
 'Bical',
 'Black Monukka',
 'Black Muscat',
 'Blatina',
 'Blauburger',
 'Blauburgunder',
 'Blauer Portugieser',
 'Blaufränkisch',
 'Bobal',
 'Bombino Bianco',
 'Bombino Nero',
 'Bonarda',
 'Bovale',
 'Boğazkere',
 'Brachetto',
 'Braucol',
 'Bual',
 'Canaiolo',
 'Cannonau',
 'Caprettone',
 'Carcajolu',
 'Carignan',
 'Carignan-Grenache',
 'Carignan-Syrah',
 'Carignane',
 'Carignano',
 'Car

In [12]:
def simple_get(url):
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)


In [13]:
raw_html = simple_get(
    'http://www.vivc.de/index.php?r=cultivarname%2Findex&CultivarnameSearch%5Bcultivarnames%5D=&CultivarnameSearch%5Bcultivarnames%5D=cultivarn&CultivarnameSearch%5Btext%5D={}'
    .format("chardonnay"))

html = BeautifulSoup(raw_html, 'html.parser')

In [22]:
out = []
count = 0
td = html.select('td')
for i, td in enumerate(td):
    try:
        value = td['data-col-seq']
        if value == "4":
            out[count]['skin'] = td.text
            count += 1
        elif value == "0":
            out.append({'cultivar' : td.text})
        elif value == "1":
            out[count]['prime'] = td.text
    except:
        pass

In [23]:
out

[{'cultivar': 'CHARDONNAY', 'prime': 'CHARDONNAY BLANC', 'skin': 'BLANC'},
 {'cultivar': 'CHARDONNAY BIJELI',
  'prime': 'CHARDONNAY BLANC',
  'skin': 'BLANC'},
 {'cultivar': 'CHARDONNAY BLANC',
  'prime': 'CHARDONNAY BLANC',
  'skin': 'BLANC'},
 {'cultivar': 'CHARDONNAY BLANC MUSQUE',
  'prime': 'CHARDONNAY BLANC MUSQUE',
  'skin': 'BLANC'},
 {'cultivar': 'CHARDONNAY BLANC SEEDLESS',
  'prime': 'CHARDONNAY BLANC SEEDLESS',
  'skin': 'BLANC'},
 {'cultivar': 'CHARDONNAY MUSQUE',
  'prime': 'CHARDONNAY BLANC MUSQUE',
  'skin': 'BLANC'},
 {'cultivar': 'CHARDONNAY NOIR', 'prime': 'CHARDONNAY NOIR', 'skin': 'NOIR'},
 {'cultivar': 'CHARDONNAY ROSA', 'prime': 'CHARDONNAY ROSE', 'skin': 'ROSE'},
 {'cultivar': 'CHARDONNAY ROSE', 'prime': 'CHARDONNAY ROSE', 'skin': 'ROSE'},
 {'cultivar': 'CHARDONNAY VERT', 'prime': 'VICLAIR BLANC', 'skin': 'BLANC'}]